In [66]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
open("/content/drive/My Drive/Colab Notebooks/file.out", "a")

<_io.TextIOWrapper name='/content/drive/My Drive/Colab Notebooks/file.out' mode='a' encoding='UTF-8'>

In [4]:
!pip install split-folders

In [5]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Installa TensorFlow
try:
  # %tensorflow_version solo existe in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import os
import tensorflow as tf
import split_folders
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from datetime import datetime

## ------------------------------------------------------------------------------------ ##
                                 # Data loader #
## ------------------------------------------------------------------------------------ ##

# fix the seed for random operations to make experiments reproducible
seed = 123
tf.random.set_seed(seed)

cwd = os.getcwd()
# path to dataset
dataset_dir = os.path.join('/content/drive/My Drive/Colab Notebooks', 'Classification_Dataset')

train_dir = os.path.join(dataset_dir, 'training')
test_dir = os.path.join(dataset_dir, 'test')

# input image dimensions
img_w = 256
img_h = 256

# number of input channels (color space)
channels = 3  # rgb

# input shape
input_shape = (img_h, img_w, channels)

class_list = [
    'owl',  # 0
    'galaxy',  # 1
    'lightning',  # 2
    'wine-bottle',  # 3
    't-shirt',  # 4
    'waterfall',  # 5
    'sword',  # 6
    'school-bus',  # 7
    'calculator',  # 8
    'sheet-music',  # 9
    'airplanes',  # 10
    'lightbulb',  # 11
    'skyscraper',  # 12
    'mountain-bike',  # 13
    'fireworks',  # 14
    'computer-monitor',  # 15
    'bear',  # 16
    'grand-piano',  # 17
    'kangaroo',  # 18
    'laptop'  # 19
]

num_classes = len(class_list) # 20

# number of training samples to feed the NN at each training step
batch_size = 16 # 8, 32, 64             # training size: 1247 samples
                                        # batch size: 16 samples/iteration
                                        # more or less 78 iterations/epochh



# Create image generators from directory
# --------------------------------------
def setup_data_generator(with_data_augmentation=True, create_test_generator=False):
    # the data, split between train and test sets

    # NOTE: splitting is done with 'flow_from_directory(…, subset=training/validation)
    # The fixed random seed is enough to reproduce the splitting.

    # fraction of images reserved for validation
    valid_split = 0.2

    # define data augmentation configuration
    apply_data_augmentation = with_data_augmentation
    if apply_data_augmentation:

        train_data_gen = ImageDataGenerator(rescale=1. / 255,  # every pixel value from range [0,255] -> [0,1]
                                            shear_range=0.2,
                                            zoom_range=0.2,
                                            rotation_range=45,
                                            horizontal_flip=True,
                                            vertical_flip=True,
                                            validation_split=valid_split)

    else:
        train_data_gen = ImageDataGenerator(rescale=1. / 255,
                                            validation_split=valid_split)

    print('\ntrain_gen ... ')
    train_generator = train_data_gen.flow_from_directory(train_dir,
                                                  subset='training',  # subset of data
                                                  batch_size=batch_size,
                                                  target_size=(img_w, img_h),  # images are automatically resized
                                                  color_mode='rgb',
                                                  classes=class_list,
                                                  class_mode='categorical',
                                                  shuffle=True,
                                                  seed=seed)

    print('\nvalid_gen ... ')
    valid_generator = train_data_gen.flow_from_directory(train_dir,
                                                  subset='validation',
                                                  batch_size=batch_size,
                                                  target_size=(img_w, img_h),
                                                  color_mode='rgb',
                                                  classes=class_list,
                                                  class_mode='categorical',
                                                  shuffle=False,
                                                  seed=seed)

    return train_generator, valid_generator

    
# Visualize accuracy and loss over time
# -------------------------------------
def visualize_performance(trained_model):
    # plt.plot(trained_model.history)

    accuracy = trained_model.history['accuracy']
    validation_accuracy = trained_model.history['val_accuracy']
    loss = trained_model.history['loss']
    validation_loss = trained_model.history['val_loss']

    epochs = range(len(accuracy))

    # Visualize History for Loss.
    plt.title('Model loss')
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, validation_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend(['training', 'validation'], loc='upper right')
    plt.show()

    # # Visualize History for Accuracy.
    plt.title('Model accuracy')
    plt.plot(epochs, accuracy, 'b', label='Training acc')
    plt.plot(epochs, validation_accuracy, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend(['training', 'validation'], loc='lower right')
    plt.show()


# Compute predictions (probabilities -- the output of the last layer)
# -------------------------------------------------------------------
def generate_predictions(model, model_name):
    results = {}
    results_str = {}

    image_filenames = next(os.walk(test_dir))[2]  # s[:10] predict until 10th image

    for filename in image_filenames:
        img = Image.open(os.path.join(test_dir, filename)).convert('RGB')  # open as RGB
        img = img.resize((img_h, img_w))  # target size

        # data_normalization
        img_array = np.array(img)  #
        img_array = img_array * 1. / 255  # normalization
        img_array = np.expand_dims(img_array, axis=0)  # to fix dims of input in the model

        print("prediction for {}...".format(filename))
        predictions = model.predict(img_array)

        # Get predicted class as the index corresponding to the maximum value in the vector probability
        predicted_class = np.argmax(predictions, axis=-1)  # multiple categories
        predicted_class = predicted_class[0]

        results[filename] = predicted_class
        results_str[filename] = class_list[predicted_class]

    create_csv(results, model_name)

    # Prints the nicely formatted dictionary
    from pprint import pprint
    pprint(results_str)

    print('Num. of labeled images', results.__len__())


# Create submission csv file
# --------------------------
def create_csv(results, model_name):
    print("\nGenerating submission csv ... ")

    # save on a different dir according to the classifier used
    results_dir = 'image_classification/submissions/' + model_name

    # If directory for the classifier does not exist, create
    if not os.path.exists(results_dir):
        os.makedirs(results_dir)

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        fieldnames = 'Id,Category'
        f.write(fieldnames + '\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

# Save model
# -------------------------------
def save_model(path, model, model_type):
    model_filename = os.path.join(path, "model_"+model_type+"-"+datetime.now().strftime('%b%d_%H-%M-%S') + ".json")
    model_json = model.to_json()
    json_file = open(model_filename, "a")
    json_file.write(model_json)
        
    # serialize weights to HDF5
    model.save_weights(model_filename+".h5")

############################################################################################

TensorFlow 2.x selected.


Using TensorFlow backend.


In [6]:
import tensorflow as tf
from keras import activations

# Data loader
# -----------
train_generator, valid_generator = setup_data_generator()

epochs = 150

transfer_learning = True
cnn = True
cnn_eval_output = ""
tl_eval_output = ""



train_gen ... 
Found 1247 images belonging to 20 classes.

valid_gen ... 
Found 307 images belonging to 20 classes.


In [24]:
###############################################
##   Transfer learning
if transfer_learning:

    # build the VGG16 network
    # ------------------------
    vgg = tf.keras.applications.VGG16(weights='imagenet',
                                      include_top=False,
                                      input_shape=input_shape)

    vgg.summary()
#     print("vgg.layers", vgg.layers)

    model_name = 'CNN+TF'

    # build a classifier model to put on top of the convolutional model

    # Two types of transfer learning: feature extraction and fine-tuning
    fine_tuning = True

    if fine_tuning:
        freeze_until = 8  # layer from which we want to fine-tune

        # set the first freeze_until layers (up to the last conv block => depth = 5)
        # to non-trainable (weights will not be updated)
        for layer in vgg.layers[:freeze_until]:
            layer.trainable = False

    else:
        vgg.trainable = False

    model = tf.keras.Sequential()
    model.add(vgg)
    model.add(tf.keras.layers.Flatten())

    # dense layers
    model.add(tf.keras.layers.Dense(units=512, activation='relu'))
    model.add(tf.keras.layers.Dense(units=512, activation='relu'))
    model.add(tf.keras.layers.Dense(units=512, activation='relu'))
    model.add(tf.keras.layers.Dense(units=512, activation='relu'))
    model.add(tf.keras.layers.Dense(units=512, activation='relu'))
    model.add(tf.keras.layers.Dense(units=512, activation='relu'))

    # final layer with softmax activation
    model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

    # Visualize created model as a table
    # model.summary()

    # Visualize initialized weights
    # print("model.weights", model.weights)

    # Prepare the model for training
    # ------------------------------
    loss = tf.keras.losses.CategoricalCrossentropy()

    # learning rate
    lr = 1e-4
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

    metrics = ['accuracy']  # validation metrics to monitor

    model.compile(optimizer=optimizer,
                  loss=loss,
                  metrics=metrics)

    step_size_train = train_generator.n // train_generator.batch_size
    trained_model = model.fit_generator(generator=train_generator,
                                        steps_per_epoch=step_size_train,
                                        epochs=epochs)

#     print('\nhistory dict:', trained_model.history)


    # Model evaluation
    # ----------------

    eval_out = model.evaluate_generator(valid_generator,
                                        steps=len(valid_generator),
                                        verbose=0)

    print('eval_out', eval_out)
    
    # /content/drive/My Drive/Colab Notebooks
    save_model('/content/drive/My Drive/Colab Notebooks', model, model_name)
   
    # Check Performance
    # visualize_performance(trained_model)

    # Generate predictions
    # -------------------
    generate_predictions(model, model_name)
    

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

In [63]:
#############################################################
##   CNN
if cnn:
    class CNNClassifier(tf.keras.Model):

        # Define two groups of layers: feature (convolutions) and classification (dense)

        def __init__(self, depth, num_filters, pool_size, kernel_size, num_classes):
            super(CNNClassifier, self).__init__()

            ## Convolution layers ##
            self.feature_extractor = tf.keras.Sequential()

            for i in range(depth):
                self.feature_extractor.add(ConvBlock(num_filters, pool_size, kernel_size))
                num_filters *= 2

            ## Flatten layer to feed data to fully connected layers ##
            self.flatten = tf.keras.layers.Flatten()  # output of a convolutional layer is a n-D tensor

            ## Classification layers ##
            self.classifier = tf.keras.Sequential()

            # Fully connected
            self.classifier.add(tf.keras.layers.Dense(units=512,
                                                      activation='relu'))
            self.classifier.add(tf.keras.layers.Dense(units=512,
                                                      activation='relu'))
            self.classifier.add(tf.keras.layers.Dense(units=512,
                                                      activation='relu'))
            self.classifier.add(tf.keras.layers.Dropout(0.1))

            # output layer with one unit for each class.
            # Use softmax activation because it's a non-binary classification problem
            self.classifier.add(tf.keras.layers.Dense(units=num_classes,
                                                      activation='softmax'))

        # create complete model = Sequential(feature_layers + classification_layers)
        def call(self, inputs):
            x = self.feature_extractor(inputs)
            x = self.flatten(x)
            x = self.classifier(x)
            return x


    # Create convolutional layer
    class ConvBlock(tf.keras.Model):

        def __init__(self, num_filters, pool_size, kernel_size):
            super(ConvBlock, self).__init__()

            self.conv2d = tf.keras.layers.Conv2D(filters=num_filters,
                                                 kernel_size=kernel_size,  # (3, 3)
                                                 # how much you move your filter when doing convolution
                                                 strides=(1, 1),
                                                 # 0 pad the input such that the output
                                                 # has the same dimensions as the original input
                                                 padding='same')

            self.activation = tf.keras.layers.ReLU()
            self.pooling = tf.keras.layers.MaxPool2D(pool_size=pool_size)  # (2, 2))
            # self.dropout.add(tf.keras.layers.Dropout(0.25))

        def call(self, inputs):
            x = self.conv2d(inputs)
            x = self.activation(x)
            x = self.pooling(x)
            # x = self.dropout(x)
            return x

        
    # Create CNN model
    # ------------
    model_name = 'CNN'

    # depth of the input volume i.e. different color channels of an image
    depth = 5

    # number of convolutional filter kernels to use
    #  weights where each is used for a convolution: trainable variables defining the filter.
    num_filters = 32

    # size of pooling area for max pooling
    pool_size = 2

    # convolution kernel size
    kernel_size = 3

    # Create Model instance
    model = CNNClassifier(depth=depth,
                      num_filters=num_filters,
                      pool_size=pool_size,
                      kernel_size=kernel_size,
                      num_classes=num_classes)

    # Build Model (Required)
    model.build(input_shape=(None, img_h, img_w, channels))

    # Visualize created model as a table
    # model.feature_extractor.summary()
    # Visualize initialized weights
    # print('initial model weights', model.weights)

    # Prepare the model for training
    # ------------------------------
    loss = tf.keras.losses.CategoricalCrossentropy()

    lr = 1e-4  # learning rate
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

    metrics = ['accuracy']  # validation metrics to monitor

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    # Train the model
    # ---------------
    with_early_stopping = True

    callbacks = []
    if with_early_stopping:
        callbacks.append(
            tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                             patience=epochs * 0.2))

    trained_model = model.fit_generator(generator=train_generator,
                                        epochs=epochs,
                                        steps_per_epoch=len(train_generator),
                                        validation_data=valid_generator,
                                        validation_steps=len(valid_generator))

    # Model evaluation
    # ----------------
    # model.load_weights('/path/to/checkpoint')  # use this if you want to restore saved model

    cnn_eval_output = model.evaluate_generator(valid_generator,
                                                steps=len(valid_generator),
                                                verbose=0)

    print('eval_out', cnn_eval_output)
    

Epoch 1/150
78/78 [==============================] - 38s 493ms/step - loss: 2.9514 - accuracy: 0.0678 - val_loss: 2.8623 - val_accuracy: 0.0879
Epoch 2/150
78/78 [==============================] - 35s 445ms/step - loss: 2.7945 - accuracy: 0.1241 - val_loss: 2.7075 - val_accuracy: 0.1401
Epoch 3/150
78/78 [==============================] - 35s 444ms/step - loss: 2.6456 - accuracy: 0.1528 - val_loss: 2.5625 - val_accuracy: 0.2182
Epoch 4/150
78/78 [==============================] - 35s 451ms/step - loss: 2.5501 - accuracy: 0.1782 - val_loss: 2.5293 - val_accuracy: 0.2280
Epoch 5/150
78/78 [==============================] - 35s 443ms/step - loss: 2.3871 - accuracy: 0.2281 - val_loss: 2.3734 - val_accuracy: 0.2476
Epoch 6/150
78/78 [==============================] - 35s 448ms/step - loss: 2.2972 - accuracy: 0.2642 - val_loss: 2.2199 - val_accuracy: 0.3257
Epoch 7/150
78/78 [==============================] - 35s 452ms/step - loss: 2.1674 - accuracy: 0.3215 - val_loss: 2.1964 - val_accuracy:

In [0]:
model.classifier.to_json()
backup_model = model

In [74]:
model.summary()

model_filename = os.path.join('/content/drive/My Drive/Colab Notebooks/models', "model_"+model_name +".json")

model_json = model.classifier.to_json()
json_file = open(model_filename, "a")
json_file.write(model_json)
    
# # serialize weights to HDF5
model.save_weights(model_filename+".h5")


model.save(model_filename + '.tf', save_format="tf")
model.save_weights(model_filename + '.weights', save_format="tf")
# Generate predictions
# -------------------
# generate_predictions(model, model_name)

Model: "cnn_classifier_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_4 (Sequential)    multiple                  1568576   
_________________________________________________________________
flatten_3 (Flatten)          multiple                  0         
_________________________________________________________________
sequential_5 (Sequential)    multiple                  17313300  
Total params: 18,881,876
Trainable params: 18,881,876
Non-trainable params: 0
_________________________________________________________________
INFO:tensorflow:Assets written to: /content/drive/My Drive/Colab Notebooks/models/model_CNN.json.tf/assets


In [75]:
# dir(model)
# loaded_model = tf.saved_model.load(model_filename+'.tf')
# dir(loaded_model)
# new_model.compile(loss='sparse_categorical_crossentropy',
#                   optimizer=keras.optimizers.RMSprop())

# # This initializes the variables used by the optimizers,
# # as well as any stateful metric variables
# new_model.train_on_batch(x_train[:1], y_train[:1])

new_model = CNNClassifier(depth=depth,
                      num_filters=num_filters,
                      pool_size=pool_size,
                      kernel_size=kernel_size,
                      num_classes=num_classes)

# Build Model (Required)
new_model.build(input_shape=(None, img_h, img_w, channels))

loss = tf.keras.losses.CategoricalCrossentropy()
lr = 1e-4  # learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
metrics = ['accuracy']  # validation metrics to monitor

new_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

# Train the model
# ---------------
with_early_stopping = True

callbacks = []
if with_early_stopping:
    callbacks.append(
        tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=epochs * 0.2))

# train on 0 epochs
model.fit_generator(generator=train_generator,
                                    epochs=0,
                                    steps_per_epoch=len(train_generator),
                                    validation_data=valid_generator,
                                    validation_steps=len(valid_generator))

# Load the state of the old model
new_model.load_weights(model_filename + '.weights')

['_TF_MODULE_IGNORED_PROPERTIES',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_activity_regularizer',
 '_add_inbound_node',
 '_add_unique_metric_name',
 '_add_variable_with_custom_getter',
 '_assert_compile_was_called',
 '_assert_weights_created',
 '_autocast',
 '_base_init',
 '_build_model_with_inputs',
 '_cache_output_metric_attributes',
 '_call_accepts_kwargs',
 '_call_arg_was_passed',
 '_call_fn_args',
 '_callable_losses',
 '_check_call_args',
 '_check_trainable_weights_consistency',
 '_checkpoint_dependencies',
 '_clear_losses',
 '_collect_input_masks',
 '_compile_distribution',
 '_compile_eagerly',
 '_compile_from_inputs',
 '_compile_metric_func

In [16]:

print('CNN Evaluation output', cnn_eval_output)
print('Transfer Learning Evaluation output', tl_eval_output)

CNN Evaluation output [3.7348770022392275, 0.52117264]
Transfer Learning Evaluation output 


**Transfer learning using our CNN model**

In [76]:
#load the entire model and check everything is correct 
from tensorflow.keras.models import load_model
from tensorflow.keras.models import model_from_json

cnn_model = new_model
cnn_model.summary()


Model: "cnn_classifier_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_6 (Sequential)    multiple                  1568576   
_________________________________________________________________
flatten_4 (Flatten)          multiple                  0         
_________________________________________________________________
sequential_7 (Sequential)    multiple                  17313300  
Total params: 18,881,876
Trainable params: 18,881,876
Non-trainable params: 0
_________________________________________________________________


In [77]:

model_name = 'Custom-CNN-TF'

# build a classifier model to put on top of the convolutional model

# Two types of transfer learning: feature extraction and fine-tuning
fine_tuning = True

if fine_tuning:
    freeze_until = 8  # layer from which we want to fine-tune

    # set the first freeze_until layers (up to the last conv block => depth = 5)
    # to non-trainable (weights will not be updated)
    for layer in cnn_model.layers[:freeze_until]:
        layer.trainable = False

else:
    cnn_model.trainable = False

model = tf.keras.Sequential()
model.add(cnn_model)
model.add(tf.keras.layers.Flatten())

# dense layers
model.add(tf.keras.layers.Dense(units=512, activation='relu'))
model.add(tf.keras.layers.Dense(units=512, activation='relu'))
model.add(tf.keras.layers.Dense(units=512, activation='relu'))
model.add(tf.keras.layers.Dense(units=512, activation='relu'))
model.add(tf.keras.layers.Dense(units=512, activation='relu'))
model.add(tf.keras.layers.Dense(units=512, activation='relu'))

# final layer with softmax activation
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

# Visualize created model as a table
# model.summary()

# Visualize initialized weights
# print("model.weights", model.weights)

# Prepare the model for training
# ------------------------------
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

metrics = ['accuracy']  # validation metrics to monitor

model.compile(optimizer=optimizer,
                loss=loss,
                metrics=metrics)

step_size_train = train_generator.n // train_generator.batch_size
trained_model = model.fit_generator(generator=train_generator,
                                    steps_per_epoch=step_size_train,
                                    epochs=epochs)

#     print('\nhistory dict:', trained_model.history)


# Model evaluation
# ----------------

eval_out = model.evaluate_generator(valid_generator,
                                    steps=len(valid_generator),
                                    verbose=0)

print('eval_out', eval_out)

# # /content/drive/My Drive/Colab Notebooks
# save_model('/content/drive/My Drive/Colab Notebooks', model, model_name)

# # Check Performance
# # visualize_performance(trained_model)

# # Generate predictions
# # -------------------
# generate_predictions(model, model_name)


Epoch 1/150
77/77 [==============================] - 30s 389ms/step - loss: 2.5613 - accuracy: 0.5774
Epoch 2/150
77/77 [==============================] - 29s 378ms/step - loss: 0.3730 - accuracy: 0.9493
Epoch 3/150
77/77 [==============================] - 30s 383ms/step - loss: 0.0603 - accuracy: 0.9902
Epoch 4/150
77/77 [==============================] - 29s 380ms/step - loss: 0.0362 - accuracy: 0.9889
Epoch 5/150
77/77 [==============================] - 30s 387ms/step - loss: 0.0733 - accuracy: 0.9884
Epoch 6/150
77/77 [==============================] - 30s 385ms/step - loss: 0.0273 - accuracy: 0.9935
Epoch 7/150
77/77 [==============================] - 30s 387ms/step - loss: 0.0643 - accuracy: 0.9891
Epoch 8/150
77/77 [==============================] - 29s 381ms/step - loss: 0.0304 - accuracy: 0.9904
Epoch 9/150
77/77 [==============================] - 30s 389ms/step - loss: 0.0701 - accuracy: 0.9859
Epoch 10/150
77/77 [==============================] - 29s 381ms/step - loss: 0.036

NotImplementedError: ignored

In [80]:
# Generate predictions
# -------------------
generate_predictions(model, model_name)


prediction for IMG_1589.jpg...
prediction for IMG_299.jpg...
prediction for IMG_1307.jpg...
prediction for IMG_264.jpg...
prediction for IMG_1501.jpg...
prediction for IMG_2017.jpg...
prediction for IMG_455.jpg...
prediction for IMG_329.jpg...
prediction for IMG_815.jpg...
prediction for IMG_664.jpg...
prediction for IMG_113.jpg...
prediction for IMG_682.jpg...
prediction for IMG_59.jpg...
prediction for IMG_497.jpg...
prediction for IMG_1382.jpg...
prediction for IMG_1322.jpg...
prediction for IMG_877.jpg...
prediction for IMG_837.jpg...
prediction for IMG_1771.jpg...
prediction for IMG_117.jpg...
prediction for IMG_1429.jpg...
prediction for IMG_1620.jpg...
prediction for IMG_1907.jpg...
prediction for IMG_1289.jpg...
prediction for IMG_420.jpg...
prediction for IMG_1219.jpg...
prediction for IMG_1197.jpg...
prediction for IMG_234.jpg...
prediction for IMG_1543.jpg...
prediction for IMG_2023.jpg...
prediction for IMG_2052.jpg...
prediction for IMG_2041.jpg...
prediction for IMG_106.j

In [81]:
print(eval_out)

[3.801963493973017, 0.5537459]
